[![CyVers](https://i.imgur.com/yyhmZET.png)](https://www.cyvers.ai/)

# Verify the CSV Files

> Notebook by:
> - Royi Avital Royi@cyverse.com

## Revision History

| Version | Date       | Content / Changes                        |
|---------|------------|------------------------------------------|
| 0.1.000 | 11/07/2022 | First version                            |
| 0.2.000 | 20/07/2022 | Added warnings related to `Amount [USD]` |

In [ ]:
# General Tools
import numpy as np
# import scipy as sp
import pandas as pd

# Misc
# import datetime
import os
from platform import python_version
import random

# EDA Tools
# import ppscore as pps #<! See https://github.com/8080labs/ppscore -> pip install git+https://github.com/8080labs/ppscore.git

# Machine Learning

# Ensemble Engines
# import lightgbm
# import xgboost

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# from bokeh.plotting import figure, show

# Jupyter
# from ipywidgets import interact, Dropdown, Layout
from IPython.display import display

In [ ]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

sns.set_theme() #>! Apply SeaBorn theme

In [ ]:
# Constants

DATA_FOLDER_NAME    = 'BlockChainAttacksDataSet'
DATA_FOLDER_PATTERN = 'DataSet0' #<! The '0' is needed in order to process templates as 'DataSet0xx' and not 'AttackDataSet'
DATA_FILE_EXT       = 'csv'

OUT_FOLDER_NAME = 'TMP'

PROJECT_DIR_NAME = 'CyVers'
PROJECT_DIR_PATH = os.path.join(os.getcwd()[:os.getcwd().find(PROJECT_DIR_NAME)], PROJECT_DIR_NAME) #>! Pay attention, it will create issues in cases you name the folder `CyVersMe` or anything after `CyVers`

In [ ]:
# CyVers Packages
from DataSetsAuxFun import *

In [ ]:
# Parameters
timeColLabel        = 'Transaction Time'
amountUsdColLabel   = 'Amount [USD]'
labelColLabel       = 'Label'

dataSetRotoDir = os.path.join(PROJECT_DIR_PATH, DATA_FOLDER_NAME)

verifySingleSenderId    = True
addFileNameCol          = True
genDoubleAssetsCsvFile  = True
genMultipleRecPerAttack = True
genAmountUsdOutliers    = True

amountUsdOutlierThr = 1e9

# Initial value
errorFile   = False
warningFile = False

In [ ]:
# Loading / Generating Data
lCsvFile = ExtractCsvFiles(dataSetRotoDir, folderNamePattern = DATA_FOLDER_PATTERN)
print(f'The number of file found: {len(lCsvFile)}')

# dfData = pd.read_csv(os.path.join(DATA_FOLDER_NAME, csvFileName))
dfData, dAssetFile = LoadCsvFilesDf(lCsvFile, baseFoldePath = '', verifySingleSenderId = verifySingleSenderId, addFileNameCol = addFileNameCol)
numRows, numCols = dfData.shape

print(f'The number of rows (Samples): {numRows}, The number of columns: {numCols}')
print(f'The data list of columns is: {dfData.columns} with {len(dfData.columns)} columns')

In [ ]:
dfData.info()

In [ ]:
dfData.head()

In [ ]:
dfGrpBySender   = dfData.groupby('Sender ID') #<! Basically per file if verifySingleSenderId = True
dfGrpByFileName = dfData.groupby('File Name')

### Checking for Errors

In [ ]:
# Check `Transaction Time` is valid

if pd.to_datetime(dfData[timeColLabel], errors = 'coerce').isnull().any():
    print(f'The DF Must have the {timeColLabel} column with a valid date format')
    errorFile = True

    display(dfData[['Transaction ID', 'File Name']][pd.to_datetime(dfData[timeColLabel], errors = 'coerce').isnull()])

In [ ]:
# Check the number of files matches the number of unique `Sender ID`

doubleAssets = False

numSenderId = len(dfData['Sender ID'].unique())
if len(lCsvFile) != numSenderId:
    print(f'The number of files: {len(lCsvFile)}, The number of unique \'Sender ID\': {numSenderId} which doesn\'t match')
    doubleAssets    = True
    errorFile       = True

In [ ]:
if genDoubleAssetsCsvFile and doubleAssets:
    lSenderId = []
    lFileName = []
    
    kk = 0
    
    for ii, (grpName, dfGroup) in enumerate(dfGrpBySender):
        vUniqueFileName = dfGroup['File Name'].unique()
        numUniqueFiles  = len(vUniqueFileName)
        for jj in range(numUniqueFiles):
            if jj == 0:
                lSenderId.append(dfGroup['Sender ID'].iloc[0])
                lFileName.append(vUniqueFileName[jj])
            else:
                lFileName[kk] += f', {vUniqueFileName[jj]}'
        kk += 1
    
    dfSenderIDFileName = pd.DataFrame(data = list(zip(lSenderId, lFileName)), columns = ['Sender ID', 'File Name'])
    
    dfSenderIDFileName.to_csv(os.path.join(OUT_FOLDER_NAME, 'DoubleFiles.csv'))

    # Faster way...
    # dd = dfData.groupby(['Sender ID', 'File Name'])['Sender ID'].count().reset_index(name='_count')
    # dd[dd['Sender ID'].isin(dd['Sender ID'].value_counts().loc[lambda x: x>1].reset_index()['index'])]

In [ ]:
# Identifying the cases with multiple `Receiver ID` for the same file
multipleReceiverPerHack = False
for ii, (grpName, dfGroup) in enumerate(dfGrpByFileName):
    if len(dfGroup[dfGroup['Label'] == 1]['Receiver ID'].unique()) > 1:
        print('We identified cases where there are multipls `Receiver ID` in a single file')

        multipleReceiverPerHack = True
        errorFile               = True

if genMultipleRecPerAttack and multipleReceiverPerHack:
    lFileName   = []
    lReceiverId = []

    kk = 0

    for ii, (grpName, dfGroup) in enumerate(dfGrpByFileName):
        dfGroupSubSet   = dfGroup[dfGroup['Label'] == 1]
        vReceiverId     = dfGroupSubSet['Receiver ID'].unique()
        numRecId        = len(vReceiverId)
        for jj in range(numRecId):
            if jj == 0:
                lFileName.append(dfGroupSubSet['File Name'].iloc[0])
                lReceiverId.append(vReceiverId[jj])
            else:
                lReceiverId[kk] += f', {vReceiverId[jj]}'
        kk += 1
    
    dfRecIDFileName = pd.DataFrame(data = list(zip(lFileName, lReceiverId)), columns = ['File Name', 'Receiver ID'])
    
    dfRecIDFileName.to_csv(os.path.join(OUT_FOLDER_NAME, 'RecIdFiles.csv'))

In [ ]:
# Check labeled cases have an invalid `Amount [USD]` (Finder results by https://stackoverflow.com/questions/27159189)

dfAttack = dfData.loc[dfData[labelColLabel] == 1]
if (dfAttack[amountUsdColLabel].isnull().any()) or ((dfAttack[amountUsdColLabel] == 0).any()) or ((dfAttack[amountUsdColLabel] == '').any()):
    print(f'There are labeled cases which have invalid \'Amount [USD]\'')
    print(f'There are #{dfAttack[amountUsdColLabel].isnull().sum()} labeled cases NAN amount (Might be Empty)')
    print(f'There are #{(dfAttack[amountUsdColLabel] == 0).sum()} labeled cases with 0 amount')
    numEmptyCells = (dfAttack[amountUsdColLabel] == '').sum()
    print(f'There are #{numEmptyCells} labeled cases with empty amount')
    
    errorFile = True

In [ ]:
# Check for MAMA Case

for grpName, dfGroup in dfGrpBySender:
    senderId = dfGroup['Sender ID'].iloc[0]
    vHackersId = dfGroup.loc[dfGroup['Label'] == 1]['Receiver ID'].unique()
    numUniqueHackers = len(vHackersId)
    if numUniqueHackers != 1:
        print(f'The file {os.path.basename(dAssetFile[senderId])} contains more than a single hacker (MAMA)')
        errorFile = True


### Checking for Warnings

In [ ]:
# Warnings - Amount of USD

amountUsdOutliers = False
dfGrpByFileName = dfData.groupby('File Name')

if (dfData[amountUsdColLabel] > amountUsdOutlierThr).any():
    print(f'Warning: We identified {(dfData[amountUsdColLabel] > amountUsdOutlierThr).sum()} cases where the transaction `Amount [USD]` is larger than the outlier thereshold: {amountUsdOutlierThr}')

    amountUsdOutliers   = True
    warningFile         = True


if genAmountUsdOutliers and amountUsdOutliers:
    lFileName   = []
    lTsxId      = []

    kk = 0

    for ii, (grpName, dfGroup) in enumerate(dfGrpByFileName):
        dfGroupSubSet   = dfGroup[dfGroup[amountUsdColLabel] > amountUsdOutlierThr]
        if len(dfGroupSubSet) > 0:
            vTsxId          = dfGroupSubSet['Transaction ID'].unique() #<! Royi: `unique()` is used here to create a numpy vector basically
            numTsxId        = len(vTsxId)
            print(f'File Name: {grpName}, Number of cases: {numTsxId}')
            for jj in range(numTsxId):
                if jj == 0:
                    lFileName.append(dfGroupSubSet['File Name'].iloc[0])
                    lTsxId.append(vTsxId[jj])
                else:
                    lTsxId[kk] += f', {vTsxId[jj]}'
            kk += 1
    
    dfRecIDFileName = pd.DataFrame(data = list(zip(lFileName, lTsxId)), columns = ['File Name', 'Transaction ID'])
    
    dfRecIDFileName.to_csv(os.path.join(OUT_FOLDER_NAME, 'AmountUsdOutlierFiles.csv'))

In [ ]:
if errorFile is True:
    print('The Data Set Is Invalid!!!')

if warningFile is True:
    print('Theere are some warning in the Data Set!')